Notebook per fare il Training e il Testing del modello per la classe di centralità 10-40 % per fare il confronto con i risultati dell'analisi standard. Nella prima parte del notebook Training e Testing, nella seconda viene stimata la Significance che si otterrebbe misurando lo yield. 

In [1]:
%pylab inline
import uproot
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import pickle
from scipy.stats import norm
from scipy import stats
%run ../Utils/analysis_utils.py

Populating the interactive namespace from numpy and matplotlib


In [2]:
training = False

# Uproot Conversion

In [3]:
df_event1=uproot.open('../../../HypertritonAnalysis/Trees/HyperTritonTree_18r.root')['_custom']['fTreeV0'].pandas.df('fCent')

In [4]:
df_event2=uproot.open('../../../HypertritonAnalysis/Trees/HyperTritonTree_18q.root')['_custom']['fTreeV0'].pandas.df('fCent')

In [5]:
Nev_1=len(df_event1.query('10<=fCent<40'))

In [6]:
Nev_2=len(df_event2.query('10<=fCent<40'))

In [7]:
Nev_10_40=Nev_2+Nev_1

In [8]:
Nev_10_40

76303477

In [9]:
df_MC = uproot.open('../../../HypertritonAnalysis/Derived_Trees/SignalTable.root')['SignalTable'].pandas.df()

In [10]:
df_data = uproot.open('../../../HypertritonAnalysis/Derived_Trees/DataTable.root')['DataTable'].pandas.df()

### Calculate pre selection efficiency

In [19]:
df_MC.columns

Index(['V0pt', 'TPCnSigmaHe3', 'DistOverP', 'InvMass', 'ArmenterosAlpha',
       'V0CosPA', 'V0Chi2', 'PiProngPt', 'He3ProngPt', 'ProngsDCA',
       'He3ProngPvDCA', 'PiProngPvDCA', 'He3ProngPvDCAXY', 'PiProngPvDCAXY',
       'NpidClustersHe3', 'TPCnSigmaPi', 'Lrec', 'Centrality', 'V0radius'],
      dtype='object')

In [20]:
cuts_presel='ProngsDCA<1.6 and He3ProngPvDCA>0.01 and He3ProngPvDCA>0.01 and V0CosPA>0.98 and 10<=Centrality<40 and 0.5<V0radius<200'

In [21]:
eff_v0=np.load('../PreSelEfficiency/eff_V0_10_40.npy')

In [22]:
eff_23=eff_v0[0]*len(df_MC.query(cuts_presel+' and 2<V0pt<3'))/len(df_MC.query('2<V0pt<3 and 10<=Centrality<40'))

In [23]:
eff_34=eff_v0[1]*len(df_MC.query(cuts_presel+' and 3<V0pt<4'))/len(df_MC.query('3<V0pt<4 and 10<=Centrality<40'))

In [24]:
eff_45=eff_v0[2]*len(df_MC.query(cuts_presel+' and 4<V0pt<5'))/len(df_MC.query('4<V0pt<5 and 10<=Centrality<40'))

In [25]:
eff_59=eff_v0[3]*len(df_MC.query(cuts_presel+' and 5<V0pt<9'))/len(df_MC.query('5<V0pt<9 and 10<=Centrality<40'))

In [26]:
eff_presel=[eff_23,eff_34,eff_45,eff_59]

In [27]:
eff_presel

[0.0, 0.0, 0.0, 0.0]

### Create ML dataframes

In [ ]:
df_18r=df_data.query('10<=Centrality<40 and 2.960<InvMass<3.050 and V0pt<=10')
df_18r = df_18r.astype('float')

In [ ]:
sig=df_MC.query(cuts_presel)

In [ ]:
bkg = df_data.query('(InvMass<2.98 or InvMass>3.005) and V0pt<=10 and 10<=Centrality<40 ')


In [ ]:
bkg['y']=0
df_MC['y']=1
df= pd.concat([df_MC,bkg])

In [ ]:
del(df_event1)
del(df_event2)
del(df_MC)
del(df_data)
del(sig)
del(bkg)

# Data preliminary

Carico i dati, defisco le variabili su cui fare Training e preparo il Training Set.

In [ ]:
df.columns

In [ ]:
training_columns = [ 'V0CosPA','ProngsDCA', 'DistOverP','ArmenterosAlpha','NpidClustersHe3','V0pt','TPCnSigmaHe3','He3ProngPvDCAXY','PiProngPvDCAXY']

In [ ]:
traindata,testdata,ytrain,ytest = train_test_split(df, df['y'], test_size=0.5)

Plotto le variabili di Training del segnale e del fondo come confronto e la matrice delle correlazioni.

In [ ]:
plot_distr(df,training_columns+['InvMass'])

In [ ]:
plot_corr(df,training_columns)

# Training

For using pre-trained models skip to the Testing part.

In [ ]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=23)
scoring = 'auc'
early_stopping_rounds = 20
num_rounds = 200
params_def = {
    # Parameters that we are going to tune.
    'max_depth':8,
    'eta':0.05,
    'gamma':0.7,
    'min_child_weight':8,
    'subsample':0.8,
    'colsample_bytree':0.9,
    'objective':'binary:logistic',
    'random_state':42,
    'silent':1,
    'nthread':4,
    'tree_method':'hist',
    'scale_pos_weight': 10}


In [ ]:
dtrain = xgb.DMatrix(data=np.asarray(traindata[training_columns]), label=ytrain, feature_names=training_columns)
#n_round = optimize_params(dtrain,params_def)
model_cent_integr = xgb.train(params_def, dtrain,num_boost_round=num_rounds) 

In [ ]:
# saving the model
pickle.dump(model_cent_integr,open("../Models/model_10_40.pkl", "wb"))

# Testing

If you skip the Training start from here.

In [ ]:
if training:
    model = model_cent_integr
else:
    model = pickle.load(open("../Models/model_10_40.pkl", "rb"))

In [ ]:
dtest = xgb.DMatrix(data=testdata[training_columns])
y_pred = model.predict(dtest,output_margin=True)
plot_roc(ytest,y_pred)

In [ ]:
plot_feature_imp(model,['gain','weight'])

In [ ]:
plot_output_train_test(model, traindata[training_columns], ytrain, testdata[training_columns], ytest, branch_names=training_columns,raw=True,log=True,location=9)
plt.ylim([10**-5,10**0])

In [ ]:
plot_output_train_test(model, traindata[training_columns], ytrain, testdata[training_columns], ytest, branch_names=training_columns,raw=True)

# BDT Efficiency 

Calcolo l'efficienza del modello in funzione dello Score.

In [ ]:
testdata.eval('Score = @y_pred',inplace=True)
efficiency_array=EfficiencyVsCuts(testdata)
plt.figure() 

# Significance Scan Vs pT

Scan della Significance Vs BDT Score negli stessi bin di pT in cui Stefano ha estratto lo yield doppio differenziale per confrontare la massima sign. ottenibile con il BDT con la sign. ottenuta con il metodo standard. Eventi in classe di centralità 10-40%.

In [ ]:
%run ../Utils/Significance_Test.py

In [ ]:
pT_list = [[2,3],[3,4], [4,5],[5,9]]
best_score_list=[]

for i in range(0,4):
    plt.figure();
    dtest = xgb.DMatrix(data=df_18r[training_columns],silent=True)
    df_18r['Score'] = model.predict(dtest,output_margin=True)
    best_score_list.append(SignificanceScan(df_18r,pT_list[i][0],pT_list[i][1],i,efficiency_array,eff_presel,Nev_10_40))
    del dtest

In [ ]:
pT_list = [[2,3],[3,4],[4,5],[5,9]]
best_score_list=[]

for i in range(0,4):
    plt.figure();
    dtest = xgb.DMatrix(data=df_18r[training_columns],silent=True)
    df_18r['Score'] = model.predict(dtest,output_margin=True)
    best_score_list.append(SignificanceScan(df_18r,pT_list[i][0],pT_list[i][1],i,efficiency_array,eff_presel,Nev_10_40,True))
    del dtest

#   Test on data

In [ ]:
dm=xgb.DMatrix(data=df_18r[training_columns],silent=True)
df_18r['Score']=model.predict(dm,output_margin=True)

In [ ]:
TestOnData(df_18r,best_score_list[0],[2,3],Nev_10_40)

In [ ]:
TestOnData(df_18r,best_score_list[1],[3,4],Nev_10_40)

In [ ]:
TestOnData(df_18r,best_score_list[2],[4,5],Nev_10_40)

In [ ]:
TestOnData(df_18r,best_score_list[3],[5,9],Nev_10_40)

# Efficiency Vs Pt

In [ ]:
eff_cuts=[]

In [ ]:
for i in range(0,4):
    index=list(np.linspace(-3,12.5,100)).index(best_score_list[i])
    eff_cuts.append(efficiency_array[index])

In [ ]:
eff_presel=np.array(eff_presel)

In [ ]:
eff_tot=eff_presel*eff_cuts
pt=[2.5,3.5,4.5,7]

In [ ]:
plt.plot(pt,eff_tot,'r.')
plt.xlabel('pt(Gev/c)')
plt.ylabel('Efficiency');